In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-8b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to walk on the Moon was Neil Armstrong, an American astronaut who stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/data-engineer-itc/job/R-63120")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Data Engineer, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select 

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Data Engineer, ITC',
  'experience': '2+ years of experience in data engineering',
  'skills': ['SQL',
   'Python',
   'PySpark',
   'Apache Airflow',
   'Databricks',
   'Snowflake',
   'AWS/GCP/Azure',
   'Spark',
   'Delta Lake',
   'Medallion architecture',
   'ETL/ELT processes',
   'Agile methodologies',
   'DevOps practices'],
  'description': 'Build and maintain ETL/ELT pipelines and reusable data components. Collaborate with peers and stakeholders to gather data requirements. Participate in code reviews and contribute to quality improvements. Monitor and troubleshoot data pipelines for performance and reliability. Support CI/CD practices in data engineering workflows.'}]

In [6]:
type(json_res)

list

In [7]:
import pandas as pd

df = pd.read_csv("C:/Users/anush/Data Science - Videos/my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [9]:
links = collection.query(query_texts=['Experience in SQL', 'Experience in Python'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [15]:
job = json_res
#job['skills']

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Anusha Rajkumar, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Start the email with: Dear Hiring Manager. Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Anusha Rajkumar, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Here is the cold email:

Dear Hiring Manager,

I came across the job description for a Data Engineer at your company and was impressed by the requirements. AtliQ, an AI & Software Consulting company, specializes in facilitating the seamless integration of business processes through automated tools. Our expertise lies in empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

I'd like to highlight that our team has extensive experience in data engineering, with a strong focus on building and maintaining ETL/ELT pipelines, reusable data components, and collaborating with peers and stakeholders to gather data requirements. Our expertise in technologies such as SQL, Python, PySpark, Apache Airflow, Databricks, Snowflake, AWS/GCP/Azure, Spark, Delta Lake, Medallion architecture, ETL/ELT processes, Agile methodologies, and DevOps practices aligns with your job requirements.

I'd like to share some of our